In [26]:
import requests
import json
import pandas as pd
import numpy as np
import googlemaps
import re

In [27]:
#pip install googlemaps

In [28]:
api_key = ' ' 
gmaps = googlemaps.Client(key=api_key )

In [29]:
def convert_duration(duration):
    matches = re.findall(r'(\d+)\s*day', duration)
    days = int(matches[0]) if matches else 0
    matches = re.findall(r'(\d+)\s*hour', duration)
    hours = int(matches[0]) if matches else 0
    matches = re.findall(r'(\d+)\s*min', duration)
    minutes = int(matches[0]) if matches else 0

    total_minutes = days * 24 * 60 + hours * 60 + minutes
    hours = total_minutes // 60
    minutes = total_minutes % 60

    return f"{hours}:{minutes:02d}"

In [30]:
with open('dests.txt', 'r') as file:
    data = []
    for line in file:
        city = line.strip()
        result = gmaps.distance_matrix(city, 'tel aviv')['rows'][0]['elements'][0]
        distance = result['distance']['text']
        duration = result['duration']['text']
        geocode_result = gmaps.geocode(city)
        latitude = geocode_result[0]['geometry']['location']['lat']
        longitude = geocode_result[0]['geometry']['location']['lng']

        duration_formatted = convert_duration(duration)

        data.append((line.strip(), distance, duration_formatted, longitude, latitude))

dis_from_israel = pd.DataFrame(data, columns=['city', 'Distance', 'Duration', 'Longitude', 'Lat'])

dis_from_israel["Duration"]= pd.to_datetime(dis_from_israel["Duration"])

In [31]:
dis_from_israel

,city,Distance,Duration,Longitude,Lat
0,Istanbul,"1,814 km",20:54,28.978359,41.008238
1,Amsterdam,"4,536 km",48:00,4.904139,52.367573
2,Valletta,"3,795 km",51:00,14.514100,35.899237
3,Basel,"4,095 km",44:00,7.588576,47.559599
4,Doha,"2,167 km",22:44,51.531040,25.285447


In [32]:
dis_from_israel.sort_values('Distance',ascending=False)


,city,Distance,Duration,Longitude,Lat
1,Amsterdam,"4,536 km",48:00,4.904139,52.367573
3,Basel,"4,095 km",44:00,7.588576,47.559599
2,Valletta,"3,795 km",51:00,14.514100,35.899237
4,Doha,"2,167 km",22:44,51.531040,25.285447
0,Istanbul,"1,814 km",20:54,28.978359,41.008238


##3 cities furthest from Tel Aviv

In [33]:
dis_from_israel.sort_values('Distance',ascending=False).head(3)

,city,Distance,Duration,Longitude,Lat
1,Amsterdam,"4,536 km",48:00,4.904139,52.367573
3,Basel,"4,095 km",44:00,7.588576,47.559599
2,Valletta,"3,795 km",51:00,14.514100,35.899237
